**1. Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:**

* InvoiceNo — номер транзакции
* StockCode — код товара
* Description — описание товара
* Quantity — количество единиц товара, добавленных в заказ
* InvoiceDate — дата транзакции 
* UnitPrice — цена за единицу товара
* CustomerID — id клиента
* Country — страна, где проживает клиент

**В данной задаче ссылка для считывания данных лежит в переменной path_to_file.
Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.**

In [2]:
import pandas as pd

retail = pd.read_csv('data.csv', encoding='ISO-8859-1')

retail_columns = retail.columns
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

**2. Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.**

Вам понадобится len(название_колонки) и drop_duplicates()

In [5]:
# Проверка количества дубликатов (строк, полностью повторяющих другие)
duplicate_count = retail.duplicated().sum()
# Удаление дубликатов (если они есть)
if duplicate_count > 0:
    retail = retail.drop_duplicates()
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


**3. Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).**

**Сколько всего транзакций отменили пользователи? Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.**

Прежде чем начать, попробуйте метод retail.InvoiceNo.str[0]

Далее используйте метод .unique()

И сделайте выбоку по retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

In [7]:
# Получаем все строки, где InvoiceNo начинается с 'C'
canceled_transactions = retail.loc[retail['InvoiceNo'].str[0] == 'C']

# Количество отмененных транзакций (строк)
canceled_count = canceled_transactions.shape[0]
canceled_count

9251

**5.Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).**

Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

**Подсказка:**
1. Начните с выборки retail.Country == 'Germany', группировки по 'CustomerID' и агрегации total_invoices=('InvoiceNo', 'nunique')
2. Далее из полученой выборки из предыдущего шага выподните метод: название_выборки.столбец.quantile(0.8)
3. Запишите полученные id пользователей в germany_top

In [10]:
# Фильтр по Германии
germany_data = retail[retail['Country'] == 'Germany']

# Группировка с подсчётом уникальных
customer_stats = germany_data.groupby('CustomerID').agg(
    total_invoices=('InvoiceNo', 'nunique')
).reset_index()

# 80 процентиль
n_threshold = customer_stats['total_invoices'].quantile(0.8)

# больше N транзакций
top_customers = customer_stats[customer_stats['total_invoices'] > n_threshold]

# id найденных пользователей
germany_top = top_customers['CustomerID'].tolist()
germany_top

[12471.0,
 12472.0,
 12474.0,
 12476.0,
 12481.0,
 12500.0,
 12569.0,
 12600.0,
 12619.0,
 12621.0,
 12626.0,
 12647.0,
 12662.0,
 12705.0,
 12708.0,
 12709.0,
 12712.0,
 12720.0]

**6. Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.**

Датафрейм записан в retail, нужные id пользователей – в germany_top.

**Подсказка**
Попробуйте использовать top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]

In [11]:
top_retail_germany = retail.loc[retail['CustomerID'].isin(germany_top)]
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


**7. Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?** 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [12]:
# Убрать post с группировкой по Stock
product_freq = (
    top_retail_germany[top_retail_germany['StockCode'] != 'POST']
    .groupby('StockCode')
    .agg(total_orders=('InvoiceNo', 'nunique'))
    .sort_values(by='total_orders', ascending=False)
)

# Популярный товар
most_common_product = product_freq.index[0]
most_common_count = product_freq.iloc[0]['total_orders']

print(f"Товар {most_common_product} добавлен в корзину {most_common_count} раз(а)")

Товар 22326 добавляли в корзину 62 раз(а)


**8. Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.**

Отфильтрованные данные записаны в retail.

**Подсказка**
retail['новый_столбец'] = retail.старый_столбец_1 * retail.старый_столбец_2

In [15]:
# Profit = кол-во * цена)
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']
retail['Revenue']

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Name: Revenue, Length: 541909, dtype: float64